In [1]:
import requests 
import tarfile
import os

# Create lit-pcba directory if it doesn't exist
os.makedirs("lit-pcba", exist_ok=True)

# URL for downloading the AVE_unbiased dataset
url = "http://drugdesign.unistra.fr/LIT-PCBA/Files/AVE_unbiased.tgz"
response = requests.get(url)

# Save downloaded content to a temporary .tgz file
with open("AVE_unbiased.tgz", "wb") as file:
    file.write(response.content)

# Extract contents of .tgz file to lit-pcba directory and clean up
with tarfile.open("AVE_unbiased.tgz", "r:gz") as tar:
    tar.extractall(path="lit-pcba")
    os.remove("AVE_unbiased.tgz")

/var/folders/4n/jpnfy7655cz2295dr_p23gl40000gn/T/ipykernel_41834/1853501963.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="lit-pcba")


In [2]:
# All receptor names in the dataset
receptor_names = [
    'ADRB2', 'ALDH1', 'ESR1_ago', 'ESR1_ant', 'FEN1',
    'GBA', 'IDH1', 'KAT2A', 'MAPK1', 'MTORC1',
    'OPRK1', 'PKM2', 'PPARG', 'TP53', 'VDR'
]

# Mapping of PDB IDs to ligand IDs
pdb_to_ligand_id = dict([
    ("3P0G", "P0G"), ("3PDS", "ERC"), ("3SN6", "P0G"), ("4LDE", "P0G"), ("4LDL", "XQC"),
    ("4LDO", "ALE"), ("4QKX", "35V"), ("6MXT", "K5Y"), ("4WP7", "3SR"), ("4WPN", "3ST"),
    ("4X4L", "3XG"), ("5AC2", "K9P"), ("5L2M", "6ZY"), ("5L2N", "6ZU"), ("5L2O", "6ZW"),
    ("5TEI", "M39"), ("1L2I", "ETC"), ("2B1V", "458"), ("2B1Z", "17M"), ("2P15", "EZT"),
    ("2Q70", "DC8"), ("2QR9", "HZ3"), ("2QSE", "1HP"), ("2QZO", "KN1"), ("4IVW", "1GJ"),
    ("4PPS", "ESE"), ("5DRJ", "5EU"), ("5DU5", "5G2"), ("5DUE", "5FY"), ("5DZI", "5KF"),
    ("5E1C", "5K8"), ("1XP1", "AIH"), ("1XQC", "AEJ"), ("2AYR", "L4G"), ("2IOG", "IOG"),
    ("2IOK", "IOK"), ("2OUZ", "C3D"), ("2POG", "WST"), ("2R6W", "LLB"), ("3DT3", "369"),
    ("5AAU", "XBR"), ("5FQV", "VQI"), ("5T92", "77W"), ("5UFX", "86Y"), ("6B0F", "C6V"),
    ("6CHW", "F3D"), ("5FV7", "R3Z"), ("2V3D", "NBV"), ("2V3E", "NND"), ("2XWD", "LGS"),
    ("2XWE", "AMF"), ("3RIK", "3RI"), ("3RIL", "3RK"), ("4I3K", "1BX"), ("4I3L", "1BZ"),
    ("4UMX", "VVS"), ("4XRX", "42V"), ("4XS3", "42W"), ("5DE1", "59D"), ("5L57", "6N3"),
    ("5L58", "6MX"), ("5LGE", "6VN"), ("5SUN", "70Q"), ("5SVF", "70P"), ("5TQH", "7J2"),
    ("6ADG", "9UO"), ("6B0Z", "C81"), ("5H84", "1VU"), ("5H86", "BCO"), ("5MLJ", "9ST"),
    ("1PME", "SB2"), ("2OJG", "19A"), ("3SA0", "NRA"), ("3W55", "1FM"), ("4QP3", "36Q"),
    ("4QP4", "36O"), ("4QP9", "35X"), ("4QTA", "38Z"), ("4QTE", "390"), ("4XJ0", "41B"),
    ("4ZZN", "CQ8"), ("5AX3", "5ID"), ("5BUJ", "4VB"), ("5V62", "FRZ"), ("6G9H", "ERW"),
    ("1FAP", "RAP"), ("1NSG", "RAD"), ("2FAP", "RAD"), ("3FAP", "ARD"), ("4DRH", "RAP"),
    ("4DRI", "RAP"), ("4DRJ", "RAP"), ("4FAP", "ARD"), ("4JSX", "17G"), ("4JT5", "P2X"),
    ("5GPG", "RAP"), ("6B73", "CVV"), ("3GQY", "DZG"), ("3GR4", "DYY"), ("3H6O", "D8G"),
    ("3ME3", "3SZ"), ("3U2Z", "07T"), ("4G1N", "NZT"), ("4JPG", "1OX"), ("5X1V", "7XX"),
    ("5X1W", "7Y0"), ("1ZGY", "BRL"), ("2I4J", "DRJ"), ("2P4Y", "C03"), ("2Q5S", "NZA"),
    ("2YFE", "YFE"), ("3B1M", "KRC"), ("3HOD", "ZZH"), ("3R8A", "HIG"), ("4CI5", "Y1N"),
    ("4FGY", "0W3"), ("4PRG", "072"), ("5TTO", "7KK"), ("5TWO", "7MV"), ("5Y2T", "8LX"),
    ("5Z5S", "RTE"), ("2VUK", "P83"), ("3ZME", "QC5"), ("4AGO", "P74"), ("4AGQ", "P96"),
    ("5G4O", "O80"), ("5O1I", "9GH"), ("3A2I", "TEJ"), ("3A2J", "TEJ"),
])

In [3]:
import os
import csv
import requests

def get_smiles_from_rcsb(comp_id):
    """
    Fetch SMILES string for a compound from RCSB PDB using GraphQL API
    
    Args:
        comp_id (str): PDB compound identifier
        
    Returns:
        str: Stereochemical SMILES string for the compound
    """
    resp = requests.post(
        "https://data.rcsb.org/graphql",
        json={
            "query": """
            query($id: String!) {
                chem_comp(comp_id: $id) {
                rcsb_chem_comp_descriptor { SMILES_stereo }
                }
            }
            """,
            "variables": {"id": comp_id}
        }
    )
    resp.raise_for_status()
    return resp.json()["data"]["chem_comp"]["rcsb_chem_comp_descriptor"]["SMILES_stereo"]

# Output CSV file path and column headers
output_csv = "lit-pcba_all_data.csv"
fieldnames = ["receptor", "mol_id", "smiles", "type"]

# Open output CSV file to write headers
with open(output_csv, "w", newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()

    # Process each receptor
    for receptor_name in receptor_names:
        # Process active/inactive training/validation sets
        for split in ["active_T", "active_V", "inactive_T", "inactive_V"]:
            filename = f"lit-pcba/{receptor_name}/{split}.smi"
            if not os.path.exists(filename):
                continue
            # Read SMILES and molecule IDs from .smi files
            with open(filename, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 2:
                        smiles, mol_id = parts
                        writer.writerow({
                            "receptor": str(receptor_name),
                            "mol_id": str(mol_id),
                            "smiles": str(smiles),
                            "type": str(split)
                        })

        # Process query ligands from mol2 files
        receptor_dir = f"lit-pcba/{receptor_name}"
        if os.path.exists(receptor_dir):
            for fname in os.listdir(receptor_dir):
                if fname.endswith("_ligand.mol2"):
                    # Extract PDB code and get corresponding compound ID
                    pdb_code = fname.split("_ligand.mol2")[0]
                    comp_id = pdb_to_ligand_id.get(pdb_code.upper())
                    if comp_id:
                        # Fetch SMILES from RCSB PDB
                        smiles = get_smiles_from_rcsb(comp_id)
                        if smiles:
                            writer.writerow({
                                "receptor": str(receptor_name),
                                "mol_id": str(pdb_code),
                                "smiles": str(smiles),
                                "type": "query"
                            })

In [4]:
import pandas as pd
from rdkit import Chem


# Read the CSV file containing molecule data
df = pd.read_csv("lit-pcba_all_data.csv", low_memory=False)

def canonicalize_smiles_no_stereo(smiles):
    """
    Convert SMILES string to canonical form without stereochemistry
    
    Args:
        smiles (str): Input SMILES string
    Returns:
        str or None: Canonical SMILES without stereochemistry, or None if invalid SMILES
    """
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol, isomericSmiles=False, canonical=True)


# Initialize list to store canonical SMILES
canonical_smiles = []

# Process molecules in batches to manage memory usage
batch_size = 250000
total = len(df)

# Iterate through dataframe in batches
for i in range(0, total, batch_size):
    # Get batch of SMILES and corresponding rows
    batch = df["smiles"].iloc[i:i+batch_size]
    batch_rows = df.iloc[i:i+batch_size]
    batch_canonical = []
    
    # Process each molecule in the batch
    for idx, (smi, row) in enumerate(zip(batch, batch_rows.itertuples(index=False))):
        try:
            # Generate canonical SMILES
            canon = canonicalize_smiles_no_stereo(smi)
            batch_canonical.append(canon)
        except Exception as e:
            print(f"Error processing molecule at global index {i+idx}:")
            print(f"  SMILES: {smi}")
            print(f"  Row: {row}")
            print(f"  Exception: {e}")
            batch_canonical.append(None)
            
    # Add batch results to main list
    canonical_smiles.extend(batch_canonical)
    print(f"Processed {min(i+batch_size, total)} / {total}")

# Add canonical SMILES as new column and save updated dataframe
df["canonical_smiles"] = canonical_smiles
df.to_csv("lit-pcba_all_data.csv", index=False)


[12:49:27] Conflicting single bond directions around double bond at index 7.
[12:49:27]   BondStereo set to STEREONONE and single bond directions set to NONE.


Processed 250000 / 2652106
Processed 500000 / 2652106


[12:50:37] Conflicting single bond directions around double bond at index 7.
[12:50:37]   BondStereo set to STEREONONE and single bond directions set to NONE.


Processed 750000 / 2652106


[12:51:21] Conflicting single bond directions around double bond at index 7.
[12:51:21]   BondStereo set to STEREONONE and single bond directions set to NONE.


...

Processed 2000000 / 2652106


[12:54:13] Conflicting single bond directions around double bond at index 7.
[12:54:13]   BondStereo set to STEREONONE and single bond directions set to NONE.


Processed 2250000 / 2652106


[12:54:36] Conflicting single bond directions around double bond at index 7.
[12:54:36]   BondStereo set to STEREONONE and single bond directions set to NONE.


Processed 2500000 / 2652106
Processed 2652106 / 2652106


In [5]:
import collections

# Initialize sets to track overlaps and repeats across all receptors
overall_queries_overlap = set()  # Query molecules that overlap with train/val sets
overall_queries_overlap_val = set()  # Query molecules that overlap with validation set
overall_train_val_overlaps = {"active": set(), "inactive": set()}  # Molecules in both train and val sets
overall_repeats = {group: set() for group in ["queries", "active_T", "active_V", "inactive_T", "inactive_V"]}  # Repeated molecules by group

# Process each receptor separately
receptors = df["receptor"].unique()
for receptor in receptors:
    found_any = False  # Track if any overlaps/repeats found for this receptor
    sub = df[df["receptor"] == receptor].copy()
    # Map type to group name (queries for query type)
    sub["group"] = sub["type"].apply(lambda t: "queries" if t == "query" else t)

    # Group by canonical SMILES to find overlaps
    grouped = sub.groupby("canonical_smiles")["group"].agg(lambda x: set(x.dropna()))
    grouped_type = sub.groupby("canonical_smiles")["type"].agg(lambda x: set(x.dropna()))

    # Find query molecules that overlap with train/val sets
    queries_overlap = []
    queries_overlap_val = []
    queries_overlap_examples = []
    for smi, groups in grouped.items():
        if "queries" in groups and (("active_T" in groups) or ("active_V" in groups) or ("inactive_T" in groups) or ("inactive_V" in groups)):
            queries_overlap.append(smi)
            overall_queries_overlap.add(smi)
            if ("active_V" in groups) or ("inactive_V" in groups):
                queries_overlap_val.append(smi)
                overall_queries_overlap_val.add(smi)
            # Store example rows for reporting
            rows = sub[sub["canonical_smiles"] == smi]
            example = {
                "smi": smi,
                "rows": [
                    {
                        "group": row['group'],
                        "type": row['type'],
                        "mol_id": row['mol_id']
                    }
                    for _, row in rows.iterrows()
                ]
            }
            queries_overlap_examples.append(example)

    # Find molecules that overlap between train and validation sets
    train_val_overlaps = {}
    train_val_examples = {}
    for label in ["active", "inactive"]:
        t_type = f"{label}_T"
        v_type = f"{label}_V"
        t_smiles = set(sub[sub["type"] == t_type]["canonical_smiles"].dropna())
        v_smiles = set(sub[sub["type"] == v_type]["canonical_smiles"].dropna())
        overlap = t_smiles & v_smiles
        train_val_overlaps[label] = overlap
        overall_train_val_overlaps[label].update(overlap)
        train_val_examples[label] = []
        if overlap:
            # Store example rows for reporting
            for smi in overlap:
                rows = sub[sub["canonical_smiles"] == smi]
                example = {
                    "smi": smi,
                    "rows": [
                        {
                            "group": row['group'],
                            "type": row['type'],
                            "mol_id": row['mol_id']
                        }
                        for _, row in rows.iterrows()
                    ]
                }
                train_val_examples[label].append(example)

    # Find repeated molecules within each group
    repeat_types = [
        ("query", "queries"),
        ("active_T", "active_T"),
        ("active_V", "active_V"), 
        ("inactive_T", "inactive_T"),
        ("inactive_V", "inactive_V"),
    ]
    repeats = {}
    repeat_examples = {}
    for t, group_name in repeat_types:
        smiles_list = sub[sub["type"] == t]["canonical_smiles"].dropna()
        counter = collections.Counter(smiles_list)
        repeated = [smi for smi, count in counter.items() if count > 1]
        repeats[group_name] = repeated
        overall_repeats[group_name].update(repeated)
        repeat_examples[group_name] = []
        # Store example rows for reporting (up to 3)
        for smi in repeated[:3]:
            rows = sub[(sub["type"] == t) & (sub["canonical_smiles"] == smi)]
            example = {
                "smi": smi,
                "rows": [
                    {
                        "group": row['group'],
                        "type": row['type'],
                        "mol_id": row['mol_id']
                    }
                    for _, row in rows.iterrows()
                ]
            }
            repeat_examples[group_name].append(example)

    # Generate report for this receptor
    output_lines = []
    if queries_overlap_val:
        found_any = True
        output_lines.append(f"\n{'='*80}\n[Receptor: {receptor}] Outrageous Canonical SMILES Overlaps")
        output_lines.append("-"*80)
        output_lines.append(f"For receptor {receptor}, the most outrageous thing happened: {len(queries_overlap_val)} query molecule(s) are also present in the validation set (active_V or inactive_V)!")
        output_lines.append("  Example(s) of query/active/inactive overlap (top 3):")
        for example in queries_overlap_examples[:3]:
            output_lines.append(f"    SMILES: {example['smi']}")
            for row in example['rows']:
                output_lines.append(f"      group: {row['group']}, type: {row['type']}, mol_id: {row['mol_id']}")
            output_lines.append("")
    elif queries_overlap:
        found_any = True
        output_lines.append(f"\n{'='*80}\n[Receptor: {receptor}] Outrageous Canonical SMILES Overlaps")
        output_lines.append("-"*80)
        output_lines.append(f"For receptor {receptor}, {len(queries_overlap)} query molecule(s) are also present in the training set (active_T or inactive_T).")
        output_lines.append("  Example(s) of query/active/inactive overlap (top 3):")
        for example in queries_overlap_examples[:3]:
            output_lines.append(f"    SMILES: {example['smi']}")
            for row in example['rows']:
                output_lines.append(f"      group: {row['group']}, type: {row['type']}, mol_id: {row['mol_id']}")
            output_lines.append("")

    # Report train/val overlaps
    for label in ["active", "inactive"]:
        overlap = train_val_overlaps[label]
        if overlap:
            if not found_any:
                output_lines.append(f"\n{'='*80}\n[Receptor: {receptor}] Outrageous Canonical SMILES Overlaps")
                output_lines.append("-"*80)
                found_any = True
            output_lines.append(f"For receptor {receptor}, {len(overlap)} {label} molecule(s) are in both training and validation sets.")
            output_lines.append(f"  Example(s) of {label}_T/{label}_V overlap (top 3):")
            for example in train_val_examples[label][:3]:
                output_lines.append(f"    SMILES: {example['smi']}")
                for row in example['rows']:
                    output_lines.append(f"      group: {row['group']}, type: {row['type']}, mol_id: {row['mol_id']}")
                output_lines.append("")

    # Report repeats
    for t, group_name in repeat_types:
        repeated = repeats[group_name]
        if repeated:
            if not found_any:
                output_lines.append(f"\n{'='*80}\n[Receptor: {receptor}] Outrageous Canonical SMILES Overlaps")
                output_lines.append("-"*80)
                found_any = True
            output_lines.append(f"For receptor {receptor}, {len(repeated)} repeating {group_name} molecule(s) found in the set.")
            output_lines.append(f"  Example(s) of repeating {group_name} (top 3):")
            for example in repeat_examples[group_name]:
                output_lines.append(f"    SMILES: {example['smi']}")
                for row in example['rows']:
                    output_lines.append(f"      group: {row['group']}, type: {row['type']}, mol_id: {row['mol_id']}")
                output_lines.append("")

    # Print report for this receptor if any issues found
    if found_any:
        output_lines.append("-"*80 + "\n")
        print("\n".join(output_lines))

# Check if any issues found across all receptors
overall_found = (
    len(overall_queries_overlap) > 0 or
    len(overall_queries_overlap_val) > 0 or
    len(overall_train_val_overlaps['active']) > 0 or
    len(overall_train_val_overlaps['inactive']) > 0 or
    any(len(overall_repeats[group]) > 0 for group in ['queries', 'active_T', 'inactive_T', 'active_V', 'inactive_V'])
)

# Print overall statistics if any issues found
if overall_found:
    print("\n" + "="*80)
    print("OVERALL OUTRAGEOUS STATS ACROSS ALL RECEPTORS")
    print("-"*80)
    if len(overall_queries_overlap) > 0:
        print(f"Number of query molecules also in training set (overall): {len(overall_queries_overlap)}")
    if len(overall_queries_overlap_val) > 0:
        print(f"Number of query molecules also in validation set (overall): {len(overall_queries_overlap_val)}")
    if len(overall_train_val_overlaps['active']) > 0:
        print(f"Number of active molecules in both train and val (overall): {len(overall_train_val_overlaps['active'])}")
    if len(overall_train_val_overlaps['inactive']) > 0:
        print(f"Number of inactive molecules in both train and val (overall): {len(overall_train_val_overlaps['inactive'])}")
    for group in ['queries', 'active_T', 'inactive_T', 'active_V', 'inactive_V']:
        if len(overall_repeats[group]) > 0:
            print(f"Number of repeating molecules in {group} (overall): {len(overall_repeats[group])}")
    print("="*80 + "\n")



[Receptor: ADRB2] Outrageous Canonical SMILES Overlaps
--------------------------------------------------------------------------------
For receptor ADRB2, 526 inactive molecule(s) are in both training and validation sets.
  Example(s) of inactive_T/inactive_V overlap (top 3):
    SMILES: C=CCOC(=O)C1=CC(c2ccccc2)C(CCCO)C(OCC)O1
      group: inactive_T, type: inactive_T, mol_id: 24824941
      group: inactive_V, type: inactive_V, mol_id: 24824611

    SMILES: CC(=O)OCC12CCC3C4(C)CCCC(C)(C)C4CCC3(C)C1CC(C1=CC(=O)OC1OC(C)=O)O2
      group: inactive_T, type: inactive_T, mol_id: 85200279
      group: inactive_T, type: inactive_T, mol_id: 85200277
      group: inactive_V, type: inactive_V, mol_id: 85200278

    SMILES: O=C(NC1CC=CCC(CC(=O)N(CCO)Cc2ccccc2)C(=O)N2CCCC2COC1=O)OCC1c2ccccc2-c2ccccc21
      group: inactive_T, type: inactive_T, mol_id: 85200822
      group: inactive_V, type: inactive_V, mol_id: 85200809

For receptor ADRB2, 1 repeating queries molecule(s) found in the set.
  Exam

In [6]:
from rdkit.Chem import rdFingerprintGenerator, DataStructs
import numpy as np
import pandas as pd

# Number of top most similar pairs to find for each receptor within the set of query molecules
top_n = 5
results = {}

# Iterate through each receptor
for receptor in receptor_names:
    # Get query molecules for this receptor
    query_df = df[(df['receptor'] == receptor) & (df['type'] == 'query')].copy()
    # Skip if less than 2 query molecules
    if query_df.empty or len(query_df) < 2:
        continue

    # Convert SMILES to RDKit molecules if needed
    query_mols = query_df['rdmol'].tolist() if 'rdmol' in query_df.columns else [Chem.MolFromSmiles(smi) for smi in query_df['smiles']]

    # Generate Morgan fingerprints with radius 2 and 4096 bits
    fp_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2**12)
    query_fps = [fp_generator.GetFingerprint(mol) for mol in query_mols]

    # Calculate Tanimoto similarity between all pairs
    n = len(query_fps)
    pairs = []
    mol_ids = query_df['mol_id'].tolist()
    for i in range(n):
        for j in range(i+1, n):
            # Get Tanimoto similarity between fingerprints
            sim = DataStructs.TanimotoSimilarity(query_fps[i], query_fps[j])
            pairs.append((sim, mol_ids[i], mol_ids[j]))

    # Get top N most similar pairs
    top_pairs = sorted(pairs, reverse=True)[:top_n]
    results[receptor] = top_pairs

# Print results for each receptor
for receptor, top_pairs in results.items():
    print(f"\nReceptor: {receptor}")
    print("Top 5 most similar query pairs (Tanimoto):")
    for sim, mol_id1, mol_id2 in top_pairs:
        print(f"  {mol_id1} - {mol_id2}: similarity = {sim:.4f}")



Receptor: ADRB2
Top 5 most similar query pairs (Tanimoto):
  4lde - 3p0g: similarity = 1.0000
  3sn6 - 4lde: similarity = 1.0000
  3sn6 - 3p0g: similarity = 1.0000
  4qkx - 3pds: similarity = 0.5634
  4ldo - 4ldl: similarity = 0.5366

Receptor: ALDH1
Top 5 most similar query pairs (Tanimoto):
  4wpn - 4wp7: similarity = 0.5882
  5l2m - 5l2n: similarity = 0.2794
  5l2n - 5l2o: similarity = 0.2712
  5l2m - 5l2o: similarity = 0.2258
  4x4l - 5tei: similarity = 0.2043

Receptor: ESR1_ago
Top 5 most similar query pairs (Tanimoto):
  5dzi - 5e1c: similarity = 0.4800
  2qzo - 4ivw: similarity = 0.4754
  5du5 - 5drj: similarity = 0.4000
  2b1v - 2q70: similarity = 0.3051
  5e1c - 2qr9: similarity = 0.2963

Receptor: ESR1_ant
Top 5 most similar query pairs (Tanimoto):
  2iog - 2iok: similarity = 0.6438
  1xp1 - 5ufx: similarity = 0.4286
  2r6w - 5ufx: similarity = 0.4250
  2r6w - 2ayr: similarity = 0.3780
  1xp1 - 2r6w: similarity = 0.3671

Receptor: GBA
Top 5 most similar query pairs (Tanimot

In [7]:
from rdkit.Chem import rdFMCS

# Number of top most similar pairs to find for each receptor within the set of query molecules
top_n = 5
mcs_results = {}

# Iterate through each receptor
for receptor in receptor_names:
    # Get query molecules for this receptor
    query_df = df[(df['receptor'] == receptor) & (df['type'] == 'query')].copy()
    # Skip if less than 2 query molecules
    if query_df.empty or len(query_df) < 2:
        continue

    # Remove duplicate SMILES to avoid redundant comparisons
    query_df_nodup = query_df.drop_duplicates(subset='smiles').copy()
    if query_df_nodup.empty or len(query_df_nodup) < 2:
        continue

    # Convert SMILES to RDKit molecules if needed
    query_mols = query_df_nodup['rdmol'].tolist() if 'rdmol' in query_df_nodup.columns else [Chem.MolFromSmiles(smi) for smi in query_df_nodup['smiles']]
    mol_ids = query_df_nodup['mol_id'].tolist()
    n = len(query_mols)
    pairs = []

    # Compare each pair of molecules
    for i in range(n):
        for j in range(i+1, n):
            mol1 = query_mols[i]
            mol2 = query_mols[j]
            # Find Maximum Common Substructure between the two molecules
            # completeRingsOnly=True ensures rings are matched as complete units
            # timeout=10 prevents hanging on difficult comparisons
            res = rdFMCS.FindMCS([mol1, mol2], completeRingsOnly=True, ringMatchesRingOnly=True, timeout=10)
            
            # Calculate similarity ratio based on MCS size
            if res.canceled or res.numAtoms == 0:
                ratio = 0.0
            else:
                mcs_smarts = res.smartsString
                mcs_mol = Chem.MolFromSmarts(mcs_smarts)
                if mcs_mol is not None:
                    # Calculate ratio of MCS atoms to smaller molecule's atoms
                    mcs_num_atoms = mcs_mol.GetNumAtoms()
                    min_atoms = min(mol1.GetNumAtoms(), mol2.GetNumAtoms())
                    ratio = mcs_num_atoms / min_atoms if min_atoms > 0 else 0.0
                else:
                    ratio = 0.0
            pairs.append((ratio, mol_ids[i], mol_ids[j]))

    # Get top N most similar pairs
    top_pairs = sorted(pairs, reverse=True)[:top_n]
    mcs_results[receptor] = top_pairs

# Print results for each receptor
for receptor, top_pairs in mcs_results.items():
    print(f"\nReceptor: {receptor}")
    print("Top 5 most similar query pairs (MCS ratio):")
    for ratio, mol_id1, mol_id2 in top_pairs:
        print(f"  {mol_id1} - {mol_id2}: MCS ratio = {ratio:.4f}")



Receptor: ADRB2
Top 5 most similar query pairs (MCS ratio):
  4qkx - 4ldo: MCS ratio = 1.0000
  4ldo - 4ldl: MCS ratio = 1.0000
  6mxt - 4ldo: MCS ratio = 0.9231
  4qkx - 3pds: MCS ratio = 0.9231
  3sn6 - 4ldo: MCS ratio = 0.9231

Receptor: ALDH1
Top 5 most similar query pairs (MCS ratio):
  4wpn - 4wp7: MCS ratio = 0.7143
  5l2n - 5l2o: MCS ratio = 0.7059
  5l2m - 5l2o: MCS ratio = 0.7059
  5l2m - 5l2n: MCS ratio = 0.5417
  4x4l - 5ac2: MCS ratio = 0.3636

Receptor: ESR1_ago
Top 5 most similar query pairs (MCS ratio):
  5dzi - 5e1c: MCS ratio = 1.0000
  2b1z - 2p15: MCS ratio = 1.0000
  2qzo - 4ivw: MCS ratio = 0.8750
  2b1z - 1l2i: MCS ratio = 0.7143
  2b1v - 2q70: MCS ratio = 0.6842

Receptor: ESR1_ant
Top 5 most similar query pairs (MCS ratio):
  2iog - 2iok: MCS ratio = 1.0000
  3dt3 - 2ayr: MCS ratio = 0.9259
  5fqv - 5t92: MCS ratio = 0.8148
  1xqc - 5t92: MCS ratio = 0.7667
  3dt3 - 2ouz: MCS ratio = 0.6296

Receptor: GBA
Top 5 most similar query pairs (MCS ratio):
  3ril - 3r

In [8]:
from rdkit.Chem import rdFingerprintGenerator, DataStructs

# Create Morgan fingerprint generator with radius 2 and 4096 bits
fp_generator = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2**12)

# Set minimum similarity threshold for considering two molecules similar
similarity_threshold = 0.6

for receptor in receptor_names:
    # Get active compounds from training and validation sets for this receptor
    active_T_df = df[(df['receptor'] == receptor) & (df['type'] == 'active_T')].copy()
    active_V_df = df[(df['receptor'] == receptor) & (df['type'] == 'active_V')].copy()
    if active_T_df.empty or active_V_df.empty:
        continue

    # Remove duplicate SMILES strings
    active_T_df_nodup = active_T_df.drop_duplicates(subset='smiles').copy()
    active_V_df_nodup = active_V_df.drop_duplicates(subset='smiles').copy()
    if active_T_df_nodup.empty or active_V_df_nodup.empty:
        continue

    # Convert SMILES to RDKit molecules if needed
    active_T_mols = active_T_df_nodup['rdmol'].tolist() if 'rdmol' in active_T_df_nodup.columns else [Chem.MolFromSmiles(smi) for smi in active_T_df_nodup['smiles']]
    active_V_mols = active_V_df_nodup['rdmol'].tolist() if 'rdmol' in active_V_df_nodup.columns else [Chem.MolFromSmiles(smi) for smi in active_V_df_nodup['smiles']]
    active_T_ids = active_T_df_nodup['mol_id'].tolist()
    active_V_ids = active_V_df_nodup['mol_id'].tolist()

    # Generate Morgan fingerprints for all molecules
    active_T_fps = [fp_generator.GetFingerprint(mol) for mol in active_T_mols]
    active_V_fps = [fp_generator.GetFingerprint(mol) for mol in active_V_mols]

    # Compare each training active against each validation active
    found_similar = []
    for i, afp in enumerate(active_T_fps):
        for j, vfp in enumerate(active_V_fps):
            # Calculate Tanimoto similarity between fingerprints
            sim = DataStructs.TanimotoSimilarity(afp, vfp)
            if sim > similarity_threshold:
                found_similar.append((sim, active_T_ids[i], active_V_ids[j]))

    # Print results for this receptor
    print(f"\nReceptor: {receptor} | Validation active_T vs active_V")
    print(f"Number of pairs with Tanimoto similarity > {similarity_threshold}: {len(found_similar)}")
    if found_similar:
        print(f"Pairs with Tanimoto similarity > {similarity_threshold}:")
        for sim, mol_id1, mol_id2 in sorted(found_similar, reverse=True):
            print(f"  {mol_id1} (active_T) - {mol_id2} (active_V): similarity = {sim:.4f}")
    else:
        print(f"No pairs with Tanimoto similarity > {similarity_threshold}")


Receptor: ADRB2 | Validation active_T vs active_V
Number of pairs with Tanimoto similarity > 0.6: 0
No pairs with Tanimoto similarity > 0.6

Receptor: ALDH1 | Validation active_T vs active_V
Number of pairs with Tanimoto similarity > 0.6: 323
Pairs with Tanimoto similarity > 0.6:
  17513169 (active_T) - 26663676 (active_V): similarity = 0.9796
  17514456 (active_T) - 17413119 (active_V): similarity = 0.8909
  849849 (active_T) - 851058 (active_V): similarity = 0.8852
  17514672 (active_T) - 17413119 (active_V): similarity = 0.8750
  865822 (active_T) - 866152 (active_V): similarity = 0.8636
  24781614 (active_T) - 24782909 (active_V): similarity = 0.8507
  4242197 (active_T) - 7973338 (active_V): similarity = 0.8485
  14742523 (active_T) - 26661991 (active_V): similarity = 0.8462
  4244867 (active_T) - 4241016 (active_V): similarity = 0.8421
  24828484 (active_T) - 24830679 (active_V): similarity = 0.8387
  7968537 (active_T) - 7975778 (active_V): similarity = 0.8367
  7967221 (active